# Introduction
In this notebook I will calculate various advanced statistics for individual players such as efficiency(EFF), true shooting percentage(TS%), etc.

## Notebook Objective
The main objective of this notebook is to join the two datasets (details, player_game_data) to create a new dataset advanced_player_data. 

# Setup
## Imports

In [1]:
import pandas as pd
import numpy as np
import datetime

# Parameters

In [2]:
# In
TEAM_DATA = '../../data/processed/010_details.csv'
DATA_TO_MERGE = '../../data/processed/004_player_game_data.csv'

# Out
ADVANCED_PLAYER_STATS = '../../data/processed/410_advanced_player_data.csv'

# Read in data
## Player game data
This data will be merged with team data

In [3]:
merge_df = pd.read_csv(DATA_TO_MERGE, dtype={"START_POSITION": object, "MIN": "string"})
merge_df.dtypes

SEASON                 int64
GAME_ID                int64
TEAM_ID                int64
PLAYER_ID              int64
PLAYER_NAME           object
TEAM_ABBREVIATION     object
START_POSITION        object
MIN                   string
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TO                   float64
PF                   float64
PTS                  float64
PLUS_MINUS           float64
GAME_DATE_EST         object
dtype: object

## Team data
This dataframe will get merged with merge_df to calculate the advanced statistics (EFF, USG%, etc.).

In [4]:
team_df = pd.read_csv(TEAM_DATA)
team_df.dtypes

GAME_ID         int64
TEAM_ID         int64
FGM           float64
FGA           float64
FG_PCT        float64
FG3M          float64
FG3A          float64
FG3_PCT       float64
FTM           float64
FTA           float64
FT_PCT        float64
OREB          float64
DREB          float64
REB           float64
AST           float64
STL           float64
BLK           float64
TO            float64
PF            float64
PTS           float64
PLUS_MINUS    float64
SEASON          int64
dtype: object

In [5]:
merge_df.head(1)

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GAME_DATE_EST
0,2020,12000047,1610612766,1628998,Cody Martin,CHA,F,17:06,0.0,2.0,...,2.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0,-31.0,2020-12-19


In [6]:
merge_df.tail(1)

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GAME_DATE_EST
491736,2012,11200005,1610612743,201951,Ty Lawson,DEN,NaN,27,3.0,6.0,...,2.0,2.0,6.0,2.0,0.0,6.0,1.0,8.0,NaN,2012-10-06


You can see from the dataframe segments above that `MIN` comes in different formats. The function below, `format_minutes`, converts all these rows to ints.

In [7]:
def format_minutes(row):
    minute = row['MIN'][0:2]
    if len(minute) > 1:
        if minute[1] == ":":
            return int(minute[0])
        else:
            return int(row['MIN'][0:2])
    else:
        return int(row['MIN'][0])

In [8]:
merge_df['MIN'] = merge_df.apply(lambda row: format_minutes(row), axis=1)
merge_df.head()

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GAME_DATE_EST
0,2020,12000047,1610612766,1628998,Cody Martin,CHA,F,17,0.0,2.0,...,2.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0,-31.0,2020-12-19
1,2020,12000047,1610612766,1629023,P.J. Washington,CHA,F,24,4.0,17.0,...,6.0,7.0,2.0,2.0,1.0,5.0,3.0,9.0,-2.0,2020-12-19
2,2020,12000047,1610612766,203469,Cody Zeller,CHA,C,22,5.0,8.0,...,2.0,4.0,0.0,0.0,1.0,0.0,2.0,13.0,-23.0,2020-12-19
3,2020,12000047,1610612766,1628984,Devonte' Graham,CHA,G,31,8.0,15.0,...,3.0,4.0,3.0,2.0,0.0,4.0,0.0,25.0,-7.0,2020-12-19
4,2020,12000047,1610612766,1626179,Terry Rozier,CHA,G,26,8.0,14.0,...,4.0,5.0,6.0,1.0,0.0,0.0,2.0,24.0,22.0,2020-12-19


There are some unacceptable values in the `MIN` column. No game in recent history is longer than 68 minutes and obviously a player can't play a negative number of minutes.

In [9]:
merge_df['MIN'].min(), merge_df['MIN'].max()

(-9, 96)

In [10]:
merge_df.shape

(491737, 28)

In [11]:
merge_df = merge_df[(merge_df['MIN'] < 68) & (merge_df['MIN'] > 0)]
merge_df.shape

(486559, 28)

# Merge dataframes
Now that the `MIN` column has been fixed we can merge the dataframes `merge_df` and `team_df`. We will merge these dataframes by `GAME_ID` and `TEAM_ID` to line up each players performance with his team's performance in each game.

In [12]:
new_df = pd.merge(merge_df, team_df,  how='left', left_on=['GAME_ID','TEAM_ID'], right_on = ['GAME_ID','TEAM_ID'], suffixes=(None, '_TEAM'))
new_df.columns

Index(['SEASON', 'GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ABBREVIATION', 'START_POSITION', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS',
       'GAME_DATE_EST', 'FGM_TEAM', 'FGA_TEAM', 'FG_PCT_TEAM', 'FG3M_TEAM',
       'FG3A_TEAM', 'FG3_PCT_TEAM', 'FTM_TEAM', 'FTA_TEAM', 'FT_PCT_TEAM',
       'OREB_TEAM', 'DREB_TEAM', 'REB_TEAM', 'AST_TEAM', 'STL_TEAM',
       'BLK_TEAM', 'TO_TEAM', 'PF_TEAM', 'PTS_TEAM', 'PLUS_MINUS_TEAM',
       'SEASON_TEAM'],
      dtype='object')

In [13]:
new_df.head()

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,DREB_TEAM,REB_TEAM,AST_TEAM,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM
0,2020,12000047,1610612766,1628998,Cody Martin,CHA,F,17,0.0,2.0,...,41.0,52.0,21.0,7.0,6.0,20.0,23.0,117.0,-15.0,2020
1,2020,12000047,1610612766,1629023,P.J. Washington,CHA,F,24,4.0,17.0,...,41.0,52.0,21.0,7.0,6.0,20.0,23.0,117.0,-15.0,2020
2,2020,12000047,1610612766,203469,Cody Zeller,CHA,C,22,5.0,8.0,...,41.0,52.0,21.0,7.0,6.0,20.0,23.0,117.0,-15.0,2020
3,2020,12000047,1610612766,1628984,Devonte' Graham,CHA,G,31,8.0,15.0,...,41.0,52.0,21.0,7.0,6.0,20.0,23.0,117.0,-15.0,2020
4,2020,12000047,1610612766,1626179,Terry Rozier,CHA,G,26,8.0,14.0,...,41.0,52.0,21.0,7.0,6.0,20.0,23.0,117.0,-15.0,2020


In [14]:
new_df.columns

Index(['SEASON', 'GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ABBREVIATION', 'START_POSITION', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS',
       'GAME_DATE_EST', 'FGM_TEAM', 'FGA_TEAM', 'FG_PCT_TEAM', 'FG3M_TEAM',
       'FG3A_TEAM', 'FG3_PCT_TEAM', 'FTM_TEAM', 'FTA_TEAM', 'FT_PCT_TEAM',
       'OREB_TEAM', 'DREB_TEAM', 'REB_TEAM', 'AST_TEAM', 'STL_TEAM',
       'BLK_TEAM', 'TO_TEAM', 'PF_TEAM', 'PTS_TEAM', 'PLUS_MINUS_TEAM',
       'SEASON_TEAM'],
      dtype='object')

# Usage Percentage (USG%)
Usage rate or usage percentage is the percentage of plays a player is involved in when they are on the court. The average usage percentage in the league is 20%. Good players are involved in more plays than other players so their usage percentage is generally higher. The formula to calculate usage percentage can be found [here](https://bleacherreport.com/articles/1039116-understanding-the-nba-explaining-advanced-offensive-stats-and-metrics#:~:text=usage%20rate%20(usg%25)).

In [15]:
new_df['USG%'] = 100*((new_df['FGA']+0.44*new_df['FTA']+new_df['TO']) * 48)/\
                (new_df['MIN']*(new_df['FGA_TEAM']+0.44*new_df['FTA_TEAM']+new_df['TO_TEAM']))

In [16]:
new_df[new_df['PLAYER_NAME'] == 'Luka Doncic']

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,REB_TEAM,AST_TEAM,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM,USG%
416,2020,12000036,1610612742,1629029,Luka Doncic,DAL,G,27,6.0,18.0,...,45.0,28.0,1.0,8.0,14.0,25.0,127.0,-11.0,2020,37.218638
752,2020,12000022,1610612742,1629029,Luka Doncic,DAL,G,29,9.0,20.0,...,49.0,22.0,4.0,2.0,9.0,18.0,128.0,80.0,2020,39.285970
1077,2020,12000009,1610612742,1629029,Luka Doncic,DAL,G,16,5.0,13.0,...,51.0,26.0,8.0,2.0,13.0,19.0,109.0,55.0,2020,36.020583
2197,2019,41900156,1610612742,1629029,Luka Doncic,DAL,G,41,15.0,28.0,...,42.0,20.0,5.0,3.0,12.0,15.0,97.0,-70.0,2019,39.052906
2309,2019,41900155,1610612742,1629029,Luka Doncic,DAL,G,31,6.0,17.0,...,31.0,19.0,8.0,6.0,13.0,22.0,111.0,-215.0,2019,38.750960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304399,2018,21800032,1610612742,1629029,Luka Doncic,DAL,F,36,8.0,16.0,...,42.0,29.0,8.0,8.0,16.0,28.0,140.0,20.0,2018,26.840364
304984,2018,21800013,1610612742,1629029,Luka Doncic,DAL,F,31,5.0,16.0,...,38.0,28.0,7.0,5.0,9.0,16.0,100.0,-105.0,2018,30.557961
305216,2018,11800071,1610612742,1629029,Luka Doncic,DAL,F,30,6.0,14.0,...,42.0,21.0,9.0,4.0,21.0,23.0,118.0,-25.0,2018,21.673307
305655,2018,11800047,1610612742,1629029,Luka Doncic,DAL,F,26,4.0,8.0,...,43.0,27.0,13.0,6.0,21.0,26.0,115.0,15.0,2018,21.557261


In [17]:
new_df.max().tail()

PF_TEAM                  45.0
PTS_TEAM                168.0
PLUS_MINUS_TEAM         305.0
SEASON_TEAM              2020
USG%               826.861314
dtype: object

Because some of the `MIN` data is incorrect we need to remove all `USG%` over 100

In [18]:
new_df = new_df[new_df['USG%'] <= 100]

# True Shooting Percentage (TS%)
True shooting percentage is a stat that takes in to account the greater risk and greater reward of the 3-point shot. Field goal percentage doesn't tell us the full story because it sees a player scoring 5 out of 7 2-point shots as equal to a player scoring 5 out of 7 3-point shots. These are clearly not equal in value, the first player scored 10 points whereas the second player scored 15 points. A description of the formula can be found [here](https://bleacherreport.com/articles/1039116-understanding-the-nba-explaining-advanced-offensive-stats-and-metrics#:~:text=true%20shooting%20percentage%20(ts%25)).

In [19]:
new_df['TS%'] = new_df['PTS']/( 2*(new_df['FGA']+0.44*new_df['FTA']) )
new_df[new_df['PLAYER_NAME'] == 'Luka Doncic']

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,AST_TEAM,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM,USG%,TS%
416,2020,12000036,1610612742,1629029,Luka Doncic,DAL,G,27,6.0,18.0,...,28.0,1.0,8.0,14.0,25.0,127.0,-11.0,2020,37.218638,0.455373
752,2020,12000022,1610612742,1629029,Luka Doncic,DAL,G,29,9.0,20.0,...,22.0,4.0,2.0,9.0,18.0,128.0,80.0,2020,39.285970,0.553279
1077,2020,12000009,1610612742,1629029,Luka Doncic,DAL,G,16,5.0,13.0,...,26.0,8.0,2.0,13.0,19.0,109.0,55.0,2020,36.020583,0.500000
2197,2019,41900156,1610612742,1629029,Luka Doncic,DAL,G,41,15.0,28.0,...,20.0,5.0,3.0,12.0,15.0,97.0,-70.0,2019,39.052906,0.602792
2309,2019,41900155,1610612742,1629029,Luka Doncic,DAL,G,31,6.0,17.0,...,19.0,8.0,6.0,13.0,22.0,111.0,-215.0,2019,38.750960,0.474957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304399,2018,21800032,1610612742,1629029,Luka Doncic,DAL,F,36,8.0,16.0,...,29.0,8.0,8.0,16.0,28.0,140.0,20.0,2018,26.840364,0.651303
304984,2018,21800013,1610612742,1629029,Luka Doncic,DAL,F,31,5.0,16.0,...,28.0,7.0,5.0,9.0,16.0,100.0,-105.0,2018,30.557961,0.296209
305216,2018,11800071,1610612742,1629029,Luka Doncic,DAL,F,30,6.0,14.0,...,21.0,9.0,4.0,21.0,23.0,118.0,-25.0,2018,21.673307,0.587467
305655,2018,11800047,1610612742,1629029,Luka Doncic,DAL,F,26,4.0,8.0,...,27.0,13.0,6.0,21.0,26.0,115.0,15.0,2018,21.557261,0.768443


# Efficiency (EFF)
Efficiency is a statistic which gives a numeric value to the performance of a player. It is a basic sum of positive actions for their team (points scored + rebounds + . . . ) minus negative actions (-missed shots - turnovers - . . . ). The formula for efficiency can be found [here](https://en.wikipedia.org/wiki/Efficiency_(basketball)#:~:text=Individual%20player%20efficiency%20is%20expressed,reporter%20and%20statistician%20Martin%20Manley.).

In [20]:
new_df['EFF'] = new_df['PTS']+new_df['DREB']+new_df['OREB']+\
                            new_df['AST']+new_df['STL']+new_df['BLK']-\
                            (new_df['FGA']-new_df['FGM'])-(new_df['FTA']-new_df['FTM'])-\
                            new_df['TO']
new_df.head()

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM,USG%,TS%,EFF
0,2020,12000047,1610612766,1628998,Cody Martin,CHA,F,17,0.0,2.0,...,7.0,6.0,20.0,23.0,117.0,-15.0,2020,7.077697,0.000000,1.0
1,2020,12000047,1610612766,1629023,P.J. Washington,CHA,F,24,4.0,17.0,...,7.0,6.0,20.0,23.0,117.0,-15.0,2020,38.235294,0.251678,2.0
2,2020,12000047,1610612766,203469,Cody Zeller,CHA,C,22,5.0,8.0,...,7.0,6.0,20.0,23.0,117.0,-15.0,2020,16.188624,0.731982,15.0
3,2020,12000047,1610612766,1628984,Devonte' Graham,CHA,G,31,8.0,15.0,...,7.0,6.0,20.0,23.0,117.0,-15.0,2020,26.289460,0.765931,23.0
4,2020,12000047,1610612766,1626179,Terry Rozier,CHA,G,26,8.0,14.0,...,7.0,6.0,20.0,23.0,117.0,-15.0,2020,23.632250,0.783290,30.0


In [21]:
new_df[new_df['PLAYER_NAME'] == 'Luka Doncic'].head()

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM,USG%,TS%,EFF
416,2020,12000036,1610612742,1629029,Luka Doncic,DAL,G,27,6.0,18.0,...,1.0,8.0,14.0,25.0,127.0,-11.0,2020,37.218638,0.455373,15.0
752,2020,12000022,1610612742,1629029,Luka Doncic,DAL,G,29,9.0,20.0,...,4.0,2.0,9.0,18.0,128.0,80.0,2020,39.285970,0.553279,21.0
1077,2020,12000009,1610612742,1629029,Luka Doncic,DAL,G,16,5.0,13.0,...,8.0,2.0,13.0,19.0,109.0,55.0,2020,36.020583,0.500000,10.0
2197,2019,41900156,1610612742,1629029,Luka Doncic,DAL,G,41,15.0,28.0,...,5.0,3.0,12.0,15.0,97.0,-70.0,2019,39.052906,0.602792,35.0
2309,2019,41900155,1610612742,1629029,Luka Doncic,DAL,G,31,6.0,17.0,...,8.0,6.0,13.0,22.0,111.0,-215.0,2019,38.750960,0.474957,14.0


# Change name of data frame

In [22]:
full_advanced_data = new_df.copy()

In [23]:
full_advanced_data.sort_values(by='GAME_DATE_EST').head(5)

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM,USG%,TS%,EFF
188259,2003,10300001,1610612742,1505,Tariq Abdul-Wahad,DAL,NaN,5,1.0,5.0,...,9.0,4.0,18.0,34.0,85.0,0.0,2003,44.776119,0.200000,6.0
188231,2003,10300001,1610612762,2144,Ademola Okulaja,UTA,NaN,5,1.0,1.0,...,8.0,4.0,18.0,26.0,90.0,0.0,2003,18.648019,1.000000,3.0
188232,2003,10300001,1610612762,1905,Andrei Kirilenko,UTA,NaN,5,0.0,5.0,...,8.0,4.0,18.0,26.0,90.0,0.0,2003,92.121212,0.085034,1.0
188233,2003,10300001,1610612762,1052,Ben Handlogten,UTA,NaN,5,2.0,4.0,...,8.0,4.0,18.0,26.0,90.0,0.0,2003,64.149184,0.409836,4.0
188234,2003,10300001,1610612762,2306,Carlos Arroyo,UTA,NaN,5,1.0,1.0,...,8.0,4.0,18.0,26.0,90.0,0.0,2003,9.324009,1.000000,5.0


In [24]:
full_advanced_data.sort_values(by='GAME_DATE_EST').tail(5)

,SEASON,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,START_POSITION,MIN,FGM,FGA,...,STL_TEAM,BLK_TEAM,TO_TEAM,PF_TEAM,PTS_TEAM,PLUS_MINUS_TEAM,SEASON_TEAM,USG%,TS%,EFF
57,2020,12000049,1610612737,1628381,John Collins,ATL,F,23,5.0,8.0,...,8.0,1.0,18.0,28.0,117.0,5.0,2020,19.491255,0.735294,18.0
56,2020,12000049,1610612737,1629631,De'Andre Hunter,ATL,F,26,0.0,7.0,...,8.0,1.0,18.0,28.0,117.0,5.0,2020,15.025402,0.228311,1.0
55,2020,12000048,1610612764,1630216,Cassius Winston,WAS,NaN,4,0.0,0.0,...,14.0,3.0,17.0,14.0,99.0,15.0,2020,19.957537,0.568182,1.0
68,2020,12000049,1610612737,1630233,Nathan Knight,ATL,NaN,2,1.0,1.0,...,8.0,1.0,18.0,28.0,117.0,5.0,2020,20.013342,1.000000,3.0
0,2020,12000047,1610612766,1628998,Cody Martin,CHA,F,17,0.0,2.0,...,7.0,6.0,20.0,23.0,117.0,-15.0,2020,7.077697,0.000000,1.0


In [25]:
full_advanced_data = full_advanced_data.sort_values(by='GAME_DATE_EST')

# Save dataset
Now that we have added our advanced statistics and sorted the dataframe by date we can save it.

In [26]:
full_advanced_data.to_csv(ADVANCED_PLAYER_STATS, index=False)
full_advanced_data.columns, ADVANCED_PLAYER_STATS

(Index(['SEASON', 'GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'PLAYER_NAME',
        'TEAM_ABBREVIATION', 'START_POSITION', 'MIN', 'FGM', 'FGA', 'FG_PCT',
        'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
        'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS',
        'GAME_DATE_EST', 'FGM_TEAM', 'FGA_TEAM', 'FG_PCT_TEAM', 'FG3M_TEAM',
        'FG3A_TEAM', 'FG3_PCT_TEAM', 'FTM_TEAM', 'FTA_TEAM', 'FT_PCT_TEAM',
        'OREB_TEAM', 'DREB_TEAM', 'REB_TEAM', 'AST_TEAM', 'STL_TEAM',
        'BLK_TEAM', 'TO_TEAM', 'PF_TEAM', 'PTS_TEAM', 'PLUS_MINUS_TEAM',
        'SEASON_TEAM', 'USG%', 'TS%', 'EFF'],
       dtype='object'),
 '../../data/processed/410_advanced_player_data.csv')

# Clear dataframes
This project contains many dataframes. To make sure we don't run out of memory we will delete our dataframes at the end of each notebook.

In [27]:
%reset -f